# Ueda's Oscillator
### Dow's final project for PHYS 250, Autumn 2024


This notebook will (at some point) be made interactive. For now, it is a series of code blocks which run computations and generate visualizations for the purposes of my poster.

### What I definitely want to cover
- postiion and phase space diagrams (ie time against position and time derivative against position, respectively)
    - good example [here](https://demonstrations.wolfram.com/DynamicalBehaviorOfADuffingOscillator/) and simple but nice one [here](https://demonstrations.wolfram.com/ForcedOscillatorWithDamping/)
- another good example of a short thing on the Duffing Oscillator [here](https://demonstrations.wolfram.com/DuffingOscillator/)
- Poincaré (first-return) sections
- bifurcation diagrams to analyze transition to chaos/sensitivity to initial conditions 
    - numerically compute lyapunov exponents



#### Maybe cover
- Hopf bifurcations
    - requires finding eigenevalues, so maybe pretty complicated
- animate a flow map
    